In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import Row

In [2]:
#conda update pyspark

In [3]:
#!pyspark --version

In [4]:
#spark.stop()

In [2]:
catalog_nm = "iceberg_catalog"
warehouse_path = "warehouse_loc"

conf = (
    pyspark.SparkConf()
    .setAppName("iceberg_with_filesystem")
    #pachages
    .set("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.4_2.12:1.5.2")  # Corrected coordinate
    #SQL Extensions
    .set('spark.sql.extensions', 'org.apache.iceberg.spark.extensions:IcebergSparkSessionExtensions')
    #Configuring catalog
    .set(f'spark.sql.catalog.{catalog_nm}','org.apache.iceberg.spark.SparkCatalog')
    .set(f'spark.sql.catalog.{catalog_nm}.type', 'hadoop')
    .set(f'spark.sql.catalog.{catalog_nm}.warehouse', warehouse_path)
)

#Start SparkSession
spark = SparkSession.builder.config(conf=conf).getOrCreate()
print("Spark Running")

:: loading settings :: url = jar:file:/Users/victorgalan/miniconda3/envs/iceberg_env/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/victorgalan/.ivy2/cache
The jars for the packages stored in: /Users/victorgalan/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.4_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-8289e8e6-aa77-4738-b944-1829e0acd3a9;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.4_2.12;1.5.2 in central
:: resolution report :: resolve 193ms :: artifacts dl 6ms
	:: modules in use:
	org.apache.iceberg#iceberg-spark-runtime-3.4_2.12;1.5.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: retrieving :: o

Spark Running


24/06/30 22:02:54 WARN SparkSession: Cannot use org.apache.iceberg.spark.extensions:IcebergSparkSessionExtensions to configure session extensions.
java.lang.ClassNotFoundException: org.apache.iceberg.spark.extensions:IcebergSparkSessionExtensions
	at java.net.URLClassLoader.findClass(URLClassLoader.java:387)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:418)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:351)
	at java.lang.Class.forName0(Native Method)
	at java.lang.Class.forName(Class.java:348)
	at org.apache.spark.util.Utils$.classForName(Utils.scala:225)
	at org.apache.spark.sql.SparkSession$.$anonfun$applyExtensions$1(SparkSession.scala:1294)
	at org.apache.spark.sql.SparkSession$.$anonfun$applyExtensions$1$adapted(SparkSession.scala:1292)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org

In [3]:
#Run a query
table_nm = "my_table"
spark.sql(f"""
            CREATE TABLE IF NOT EXISTS {catalog_nm}.{table_nm} (
            name STRING,
            surname STRING
            ) 
            using iceberg;
            """
         ).show()

++
||
++
++



In [7]:
#spark

In [8]:
#insert into with sql
spark.sql(
    f"""
    INSERT INTO {catalog_nm}.{table_nm} 
    VALUES ('VictoR', 'GalaN')
    """
)

DataFrame[]

In [9]:
#Insert into with dataframe api
df = spark.createDataFrame([Row(name="Sr. victor",surname="galan")])
df.writeTo(f"{catalog_nm}.{table_nm}").append()

In [10]:
#!python --version

In [4]:
#Query the table
spark.sql(
    f"""SELECT * 
    FROM {catalog_nm}.{table_nm};
    """
).show()

+----+-------+
|name|surname|
+----+-------+
+----+-------+



In [12]:
#Query the table
spark.sql("SELECT * FROM iceberg.names;").show()

AnalysisException: Unsupported data source type for direct query on files: iceberg.; line 1 pos 14

In [7]:
# Show catalogs
spark.sql("SHOW CATALOGS").show()

# Use a specific catalog (if necessary)
#spark.sql("USE CATALOG spark_catalog")

# Show databases
spark.sql("SHOW DATABASES").show()

# Show tables in a specific database
spark.sql("SHOW TABLES IN default").show()



+---------------+
|        catalog|
+---------------+
|iceberg_catalog|
|  spark_catalog|
+---------------+

+---------+
|namespace|
+---------+
|  default|
+---------+

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
+---------+---------+-----------+



In [9]:
# Show databases in the catalog
spark.sql(f"SHOW DATABASES IN {catalog_nm}").show()


+---------+
|namespace|
+---------+
+---------+



In [10]:
# Create a new database in the catalog
spark.sql(f"CREATE DATABASE {catalog_nm}.my_database")

DataFrame[]

In [13]:
# Create a new table in the database
spark.sql(f"""
    CREATE TABLE {catalog_nm}.my_database.my_table (
        id BIGINT,
        data STRING
    )
    USING iceberg
""")


DataFrame[]

In [16]:
# Show catalogs
spark.sql("SHOW CATALOGS").show()

# Use a specific catalog (if necessary)
#spark.sql("USE CATALOG spark_catalog")

# Show databases
spark.sql("SHOW DATABASES").show()

# Show tables in a specific database
spark.sql(f"SHOW TABLES IN {catalog_nm}.my_database").show()



+---------------+
|        catalog|
+---------------+
|iceberg_catalog|
|  spark_catalog|
+---------------+

+---------+
|namespace|
+---------+
|  default|
+---------+

+-----------+---------+-----------+
|  namespace|tableName|isTemporary|
+-----------+---------+-----------+
|my_database| my_table|      false|
+-----------+---------+-----------+



In [26]:
spark.sql(f"""select * 
        from {catalog_nm}.my_database.my_table
        """).show()


+---+----+
| id|data|
+---+----+
+---+----+



In [27]:
spark.sql(f"""INSERT INTO {catalog_nm}.my_database.my_table
        VALUES (1, 'hola')
        """).show()

++
||
++
++



In [28]:
spark.sql(f"""select * 
        from {catalog_nm}.my_database.my_table
        """).show()


+---+----+
| id|data|
+---+----+
|  1|hola|
+---+----+

